In [2]:
import pandas as pd

In [3]:
df = pd.read_csv("spotify_millsongdata.csv")

In [4]:
df.head(5)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [5]:
df.tail(5)

,artist,song,link,text
57645,Ziggy Marley,Good Old Days,/z/ziggy+marley/good+old+days_10198588.html,Irie days come on play \r\nLet the angels fly...
57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \r\nMore power \r\nPowe...
57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \r\nis something i'll believe \...
57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \r\nam i frightened \r\nwhere ...
57649,Zwan,Heartsong,/z/zwan/heartsong_20148991.html,come in \r\nmake yourself at home \r\ni'm a ...


In [6]:
df.shape

(57650, 4)

In [7]:
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [8]:
df =df.sample(5000).drop('link', axis=1).reset_index(drop=True)

In [9]:
df.head(10)

,artist,song,text
0,David Bowie,Andy Warhol,"(This is Andy Warhole and it's take one, take ..."
1,Kenny Chesney,Spread The Love,All God's children we got to come together \r...
2,Scorpions,10 Light Years Away,In a run down bed and breakfast \r\nA room wi...
3,Pat Benatar,Don't Happen No More,Well there used to be a time when a man was a ...
4,Frank Sinatra,Come Dance With Me,"Hey there cutes, put on your dancin' boots \r..."
5,Leo Sayer,Frankie Lee,On every corner of every avenue \r\nThe word'...
6,David Pomeranz,Undying Admiration,Sometimes late at night \r\nI sit beside our ...
7,Toto,I'll Be Over You,Some people live their dreams \r\nSome people...
8,Zebrahead,Falling Apart,Stutter step through another trap set by decep...
9,Utada Hikaru,Hotel Lobby,"She rises with the sunset \r\nShe wonders, ""W..."


In [10]:
df['text'][0]

"(This is Andy Warhole and it's take one, take one)  \r\nIt's, it's Warhol actually  \r\n(What did I say)  \r\nWhole, it's whole as in wholes  \r\n(Andy Warhol)  \r\nWah, Andy War hol, Andy War hol (he)  \r\nLike whole hub  \r\nHe  \r\nHa  \r\nAre you ready  \r\n(Yeah)  \r\nHa ha ha ha ha ha  \r\n  \r\nLike to take a cement fix  \r\nBe a standing cinema  \r\nDress my friends up just for show  \r\nSee them as they really are  \r\nPut a peephole in my brain  \r\nTwo New Pence to have a go  \r\nI'd like to be a gallery  \r\nPut you all inside my show  \r\n  \r\nAndy Warhol looks a scream  \r\nHang him on my wall  \r\nAndy Warhol, Silver Screen  \r\nCan't tell them apart at all  \r\n  \r\nAndy walking, Andy tired  \r\nAndy take a little snooze  \r\nTie him up when he's fast asleep  \r\nSend him on a pleasant cruise (hm hm hm)  \r\nWhen he wakes up on the sea  \r\nHe sure to think of me and you  \r\nHe'll think about paint and he'll think about glue  \r\nWhat a jolly boring thing to do  \r\

In [15]:
# df = df.sample(5000)

In [11]:
df.shape

(5000, 3)

Text Cleaning/ Text Preprocessing

In [12]:
df['text'] = df['text'].str.lower().replace(r'^\w\s', ' ').replace(r'\n', ' ', regex = True)

In [13]:
import nltk
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(stemming)

In [14]:
df['text'] = df['text'].apply(lambda x: tokenization(x))

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [16]:
tfidvector = TfidfVectorizer(analyzer='word',stop_words='english')
matrix = tfidvector.fit_transform(df['text'])
similarity = cosine_similarity(matrix)

In [17]:
similarity[0]

array([1.        , 0.00961662, 0.        , ..., 0.00378098, 0.00332937,
       0.01847088])

In [18]:
df[df['song'] == 'Crying Over You']

,artist,song,text


In [19]:
def recommendation(song_df):
    idx = df[df['song'] == song_df].index[0]
    distances = sorted(list(enumerate(similarity[idx])),reverse=True,key=lambda x:x[1])
    
    songs = []
    for m_id in distances[1:21]:
        songs.append(df.iloc[m_id[0]].song)
        
    return songs

In [20]:
recommendation('Crying Over You')

IndexError: index 0 is out of bounds for axis 0 with size 0

In [21]:
import pickle
pickle.dump(similarity,open('similarity.pkl','wb'))
pickle.dump(df,open('df.pkl','wb'))